A. 카카오 로컬 API (한국용 권장)

장점: 한국 주소 구성 필드(시도/시군구/동) 명확. 응답 안정적.

요구: Kakao Developers에서 REST API Key 발급.



https://developers.kakao.com/docs/latest/ko/kakaologin/rest-api

https://imweb.me/faq?mode=view&category=29&category2=47&idx=71441

https://developers.kakao.com/console/app/1317924/config

In [6]:
import pandas as pd, requests, time

REST_API_KEY = "c2c5289d7f9ea5cfa234222f18883527"
IN  = "/data1/project/yeonu/capstone/data/photo_meta.csv"
OUT = "/data1/project/yeonu/capstone/data/photo_meta_with_addr.csv"

def kakao_addr(lat, lon, key):
    h = {"Authorization": f"KakaoAK {key}"}
    # 1) coord2address 시도
    p = {"x": str(lon), "y": str(lat), "input_coord": "WGS84"}
    r = requests.get("https://dapi.kakao.com/v2/local/geo/coord2address.json",
                     headers=h, params=p, timeout=8)
    if r.status_code != 200:
        return {"_err": f"addr {r.status_code}: {r.text[:120]}"}
    docs = r.json().get("documents", [])
    # 2) 값 없으면 coord2regioncode(법정동 코드)로 폴백
    if not docs:
        p2 = {"x": str(lon), "y": str(lat), "input_coord": "WGS84"}
        r2 = requests.get("https://dapi.kakao.com/v2/local/geo/coord2regioncode.json",
                          headers=h, params=p2, timeout=8)
        if r2.status_code != 200:
            return {"_err": f"region {r2.status_code}: {r2.text[:120]}"}
        docs2 = r2.json().get("documents", [])
        if not docs2:
            return {"_err": "empty both"}
        d = docs2[0]
        return {
            "sido": d.get("region_1depth_name"),
            "sigungu": d.get("region_2depth_name"),
            "eup_myeon_dong": d.get("region_3depth_name"),
            "road_address": None,
            "address_name": d.get("address_name"),
        }
    # coord2address 성공
    a = docs[0].get("address") or docs[0].get("road_address") or {}
    raddr = docs[0].get("road_address") or {}
    return {
        "sido": a.get("region_1depth_name"),
        "sigungu": a.get("region_2depth_name"),
        "eup_myeon_dong": a.get("region_3depth_name"),
        "road_address": raddr.get("address_name"),
        "address_name": a.get("address_name") or raddr.get("address_name"),
    }

# 테스트 1건 로그 확인
print(kakao_addr(37.558444, 127.000083, REST_API_KEY))

# 일괄 처리
df = pd.read_csv(IN)
mask = df["Latitude"].notna() & df["Longitude"].notna()
rows = []
for lat, lon in df.loc[mask, ["Latitude","Longitude"]].itertuples(index=False):
    rows.append(kakao_addr(lat, lon, REST_API_KEY))
    time.sleep(0.05)  # 과한 호출 방지
res = pd.DataFrame(rows, index=df.index[mask])
df.loc[mask, res.columns] = res
df.to_csv(OUT, index=False)
print(f"저장: {OUT}")
df.head(10)


{'sido': '서울', 'sigungu': '중구', 'eup_myeon_dong': '필동3가', 'road_address': '서울특별시 중구 필동로1길 30', 'address_name': '서울 중구 필동3가 26-1'}
저장: /data1/project/yeonu/capstone/data/photo_meta_with_addr.csv


,SourceFile,FileName,Directory,Make,CapturedTime,Model,DateTimeOriginal,CreateDate,ModifyDate,ExposureTime,FNumber,ISO,Latitude,Longitude,GPSAltitude,sido,sigungu,eup_myeon_dong,road_address,address_name
0,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:27 20:12:55,iPhone 11,2025:09:27 20:12:55,2025:09:27 20:12:55,2025:09:27 20:12:55,0.125000,1.8,640,37.537031,126.945808,62.511064,서울,마포구,마포동,None,서울 마포구 마포동 253
1,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155_01.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:26 22:15:31,iPhone 11,2025:09:26 22:15:31,2025:09:26 22:15:31,2025:09:26 22:15:31,0.066667,1.8,1000,37.558444,127.000083,86.352747,서울,중구,필동3가,서울특별시 중구 필동로1길 30,서울 중구 필동3가 26-1
2,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155_02.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:26 19:55:37,iPhone 11,2025:09:26 19:55:37,2025:09:26 19:55:37,2025:09:26 19:55:37,0.033333,1.8,800,37.556553,127.001106,77.399855,서울,중구,필동3가,서울특별시 중구 필동로1길 30,서울 중구 필동3가 26-1
3,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155_03.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:26 18:51:49,iPhone 11,2025:09:26 18:51:49,2025:09:26 18:51:49,2025:09:26 18:51:49,0.500000,1.8,1000,37.559722,126.999594,78.044911,서울,중구,필동3가,서울특별시 중구 필동로1길 30,서울 중구 필동3가 26-1
4,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:29 13:10:30,iPhone 11,2025:09:29 13:10:30,2025:09:29 13:10:30,2025:09:29 13:10:30,0.008333,1.8,80,37.556975,127.003028,66.403985,서울,중구,필동3가,서울특별시 중구 필동로1길 30,서울 중구 필동3가 26-1
5,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_02.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:01 13:19:29,iPhone 11,2025:10:01 13:19:29,2025:10:01 13:19:29,2025:10:01 13:19:29,0.008333,1.8,64,37.562481,127.003928,41.175348,서울,중구,쌍림동,None,서울 중구 쌍림동 182-88
6,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_03.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:01 13:40:49,iPhone 11,2025:10:01 13:40:49,2025:10:01 13:40:49,2025:10:01 13:40:49,0.000493,1.8,32,37.562497,127.004411,44.903452,서울,중구,장충동2가,None,서울 중구 장충동2가 100-1
7,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_04.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:02 19:43:52,iPhone 11,2025:10:02 19:43:52,2025:10:02 19:43:52,2025:10:02 19:43:52,0.016667,2.4,640,37.540036,126.943778,22.070556,서울,마포구,용강동,None,서울 마포구 용강동 40-1
8,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_05.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:11 16:56:37,iPhone 11,2025:10:11 16:56:37,2025:10:11 16:56:37,2025:10:11 16:56:37,0.025000,1.8,640,37.576672,126.975197,46.970984,서울,종로구,세종로,서울특별시 종로구 효자로 12,서울 종로구 세종로 1-57
9,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_06.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:12 16:33:26,iPhone 11,2025:10:12 16:33:26,2025:10:12 16:33:26,2025:10:12 16:33:26,0.025000,1.8,400,37.540072,126.948853,30.648564,서울,마포구,도화동,None,서울 마포구 도화동 7-2


In [11]:
import pandas as pd

df = pd.read_csv("/data1/project/yeonu/capstone/data/photo_meta_with_addr.csv")

# 시·도 풀네임 매핑
sido_map = {
    "서울":"서울특별시","부산":"부산광역시","대구":"대구광역시","인천":"인천광역시",
    "광주":"광주광역시","대전":"대전광역시","울산":"울산광역시","세종":"세종특별자치시",
    "경기":"경기도","강원":"강원특별자치도","충북":"충청북도","충남":"충청남도",
    "전북":"전북특별자치도","전남":"전라남도","경북":"경상북도","경남":"경상남도",
    "제주":"제주특별자치도"
}
df["sido_full"] = df["sido"].map(sido_map).fillna(df["sido"])

# 도로명 우선, 없으면 지번
df["addr_best"] = df["road_address"].fillna(df["address_name"])

# 깔끔한 한 줄 주소
df["addr_pretty"] = (
    df["sido_full"].fillna("") + " " +
    df["sigungu"].fillna("")   + " " +
    df["eup_myeon_dong"].fillna("")
).str.strip()

df.to_csv("/data1/project/yeonu/capstone/data/photo_meta_addr_pretty.csv", index=False)
print("저장: /data1/project/yeonu/capstone/data/photo_meta_addr_pretty.csv")
df[["FileName", "CapturedTime", "Latitude","Longitude","addr_pretty","addr_best"]].head(10)


저장: /data1/project/yeonu/capstone/data/photo_meta_addr_pretty.csv


,FileName,CapturedTime,Latitude,Longitude,addr_pretty,addr_best
0,KakaoTalk_20250929_011857155.jpg,2025:09:27 20:12:55,37.537031,126.945808,서울특별시 마포구 마포동,서울 마포구 마포동 253
1,KakaoTalk_20250929_011857155_01.jpg,2025:09:26 22:15:31,37.558444,127.000083,서울특별시 중구 필동3가,서울특별시 중구 필동로1길 30
2,KakaoTalk_20250929_011857155_02.jpg,2025:09:26 19:55:37,37.556553,127.001106,서울특별시 중구 필동3가,서울특별시 중구 필동로1길 30
3,KakaoTalk_20250929_011857155_03.jpg,2025:09:26 18:51:49,37.559722,126.999594,서울특별시 중구 필동3가,서울특별시 중구 필동로1길 30
4,KakaoTalk_20251013_201618088.jpg,2025:09:29 13:10:30,37.556975,127.003028,서울특별시 중구 필동3가,서울특별시 중구 필동로1길 30
5,KakaoTalk_20251013_201618088_02.jpg,2025:10:01 13:19:29,37.562481,127.003928,서울특별시 중구 쌍림동,서울 중구 쌍림동 182-88
6,KakaoTalk_20251013_201618088_03.jpg,2025:10:01 13:40:49,37.562497,127.004411,서울특별시 중구 장충동2가,서울 중구 장충동2가 100-1
7,KakaoTalk_20251013_201618088_04.jpg,2025:10:02 19:43:52,37.540036,126.943778,서울특별시 마포구 용강동,서울 마포구 용강동 40-1
8,KakaoTalk_20251013_201618088_05.jpg,2025:10:11 16:56:37,37.576672,126.975197,서울특별시 종로구 세종로,서울특별시 종로구 효자로 12
9,KakaoTalk_20251013_201618088_06.jpg,2025:10:12 16:33:26,37.540072,126.948853,서울특별시 마포구 도화동,서울 마포구 도화동 7-2


<font color="blue">**photo_meta_addr_pretty.csv**</font> df 살펴보기

In [9]:
df

,SourceFile,FileName,Directory,Make,CapturedTime,Model,DateTimeOriginal,CreateDate,ModifyDate,ExposureTime,...,Longitude,GPSAltitude,sido,sigungu,eup_myeon_dong,road_address,address_name,sido_full,addr_best,addr_pretty
0,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:27 20:12:55,iPhone 11,2025:09:27 20:12:55,2025:09:27 20:12:55,2025:09:27 20:12:55,0.125000,...,126.945808,62.511064,서울,마포구,마포동,NaN,서울 마포구 마포동 253,서울특별시,서울 마포구 마포동 253,서울특별시 마포구 마포동
1,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155_01.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:26 22:15:31,iPhone 11,2025:09:26 22:15:31,2025:09:26 22:15:31,2025:09:26 22:15:31,0.066667,...,127.000083,86.352747,서울,중구,필동3가,서울특별시 중구 필동로1길 30,서울 중구 필동3가 26-1,서울특별시,서울특별시 중구 필동로1길 30,서울특별시 중구 필동3가
2,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155_02.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:26 19:55:37,iPhone 11,2025:09:26 19:55:37,2025:09:26 19:55:37,2025:09:26 19:55:37,0.033333,...,127.001106,77.399855,서울,중구,필동3가,서울특별시 중구 필동로1길 30,서울 중구 필동3가 26-1,서울특별시,서울특별시 중구 필동로1길 30,서울특별시 중구 필동3가
3,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20250929_011857155_03.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:26 18:51:49,iPhone 11,2025:09:26 18:51:49,2025:09:26 18:51:49,2025:09:26 18:51:49,0.500000,...,126.999594,78.044911,서울,중구,필동3가,서울특별시 중구 필동로1길 30,서울 중구 필동3가 26-1,서울특별시,서울특별시 중구 필동로1길 30,서울특별시 중구 필동3가
4,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:09:29 13:10:30,iPhone 11,2025:09:29 13:10:30,2025:09:29 13:10:30,2025:09:29 13:10:30,0.008333,...,127.003028,66.403985,서울,중구,필동3가,서울특별시 중구 필동로1길 30,서울 중구 필동3가 26-1,서울특별시,서울특별시 중구 필동로1길 30,서울특별시 중구 필동3가
5,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_02.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:01 13:19:29,iPhone 11,2025:10:01 13:19:29,2025:10:01 13:19:29,2025:10:01 13:19:29,0.008333,...,127.003928,41.175348,서울,중구,쌍림동,NaN,서울 중구 쌍림동 182-88,서울특별시,서울 중구 쌍림동 182-88,서울특별시 중구 쌍림동
6,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_03.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:01 13:40:49,iPhone 11,2025:10:01 13:40:49,2025:10:01 13:40:49,2025:10:01 13:40:49,0.000493,...,127.004411,44.903452,서울,중구,장충동2가,NaN,서울 중구 장충동2가 100-1,서울특별시,서울 중구 장충동2가 100-1,서울특별시 중구 장충동2가
7,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_04.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:02 19:43:52,iPhone 11,2025:10:02 19:43:52,2025:10:02 19:43:52,2025:10:02 19:43:52,0.016667,...,126.943778,22.070556,서울,마포구,용강동,NaN,서울 마포구 용강동 40-1,서울특별시,서울 마포구 용강동 40-1,서울특별시 마포구 용강동
8,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_05.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:11 16:56:37,iPhone 11,2025:10:11 16:56:37,2025:10:11 16:56:37,2025:10:11 16:56:37,0.025000,...,126.975197,46.970984,서울,종로구,세종로,서울특별시 종로구 효자로 12,서울 종로구 세종로 1-57,서울특별시,서울특별시 종로구 효자로 12,서울특별시 종로구 세종로
9,/data1/project/yeonu/capstone/data/photo/Kakao...,KakaoTalk_20251013_201618088_06.jpg,/data1/project/yeonu/capstone/data/photo,Apple,2025:10:12 16:33:26,iPhone 11,2025:10:12 16:33:26,2025:10:12 16:33:26,2025:10:12 16:33:26,0.025000,...,126.948853,30.648564,서울,마포구,도화동,NaN,서울 마포구 도화동 7-2,서울특별시,서울 마포구 도화동 7-2,서울특별시 마포구 도화동


In [10]:
df.columns

Index(['SourceFile', 'FileName', 'Directory', 'Make', 'CapturedTime', 'Model',
       'DateTimeOriginal', 'CreateDate', 'ModifyDate', 'ExposureTime',
       'FNumber', 'ISO', 'Latitude', 'Longitude', 'GPSAltitude', 'sido',
       'sigungu', 'eup_myeon_dong', 'road_address', 'address_name',
       'sido_full', 'addr_best', 'addr_pretty'],
      dtype='object')

좋아요.
아래는 메타데이터 CSV 각 컬럼의 **정확한 의미와 분석 시 활용 포인트**입니다.
(`exiftool` 표준 태그 기준으로 설명)

---

### 📸 **기본 파일 정보**

| 컬럼명            | 설명                   | 비고                      |
| :------------- | :------------------- | :---------------------- |
| **SourceFile** | 파일의 절대경로 (사진의 원본 위치) | 파일 추적·관리용. 원본 폴더 경로 포함. |
| **FileName**   | 파일 이름 (확장자 포함)       | 시각화나 로그에 표시할 때 사용.      |
| **Directory**  | 파일이 속한 폴더 경로         | 상위 폴더 구조 확인용.           |

---

### ⚙️ **촬영 기기 정보**

| 컬럼명       | 설명                                   | 비고            |
| :-------- | :----------------------------------- | :------------ |
| **Make**  | 제조사 (예: Apple, Samsung, Canon)       | 기기 브랜드 식별.    |
| **Model** | 기기 모델명 (예: iPhone 11, Canon EOS 80D) | 모델별 특성 분석 가능. |

---

### 🕓 **시간 관련 정보**

| 컬럼명                  | 설명                                            | 비고                   |
| :------------------- | :-------------------------------------------- | :------------------- |
| **DateTimeOriginal** | 촬영 시각 (카메라 내부 기준)                             | 가장 정확한 ‘사진 찍은 시간’.   |
| **CreateDate**       | 메타데이터 생성 시각 (보통 촬영 시각과 동일)                    | 일부 기기에서는 복사/편집 시 변경. |
| **ModifyDate**       | 파일 수정 시각 (편집, 저장 시 갱신)                        | 나중에 사진을 수정했는지 확인 가능. |
| **CapturedTime**     | `DateTimeOriginal` 또는 `CreateDate`를 통합한 대표 시각 | 시각화나 정렬용으로 추가한 컬럼.   |

---

### 📷 **촬영 설정값**

| 컬럼명              | 설명                                | 비고                           |
| :--------------- | :-------------------------------- | :--------------------------- |
| **ExposureTime** | 셔터 속도 (초 단위, 예: 0.033333 → 1/30초) | 낮은 값일수록 어두운 환경에서 흔들림 방지 어려움. |
| **FNumber**      | 조리개 값 (예: 1.8, 2.4 등)             | 낮을수록 배경 흐림(심도 얕음).           |
| **ISO**          | 감도 (예: 100, 800 등)                | 높을수록 밝게 찍히지만 노이즈 증가.         |

---

### 📍 **위치 정보 (GPS)**

| 컬럼명             | 설명                | 비고                              |
| :-------------- | :---------------- | :------------------------------ |
| **Latitude**    | 위도 (북위 +, 남위 -)   | 소수점 형태로 변환된 WGS84 좌표.           |
| **Longitude**   | 경도 (동경 +, 서경 -)   | 지도 API 입력값으로 그대로 사용 가능.         |
| **GPSAltitude** | 촬영 지점의 고도 (미터 단위) | 고층·산지 구분 가능. 일부 기기에서 부정확할 수 있음. |

---

### 📊 **활용 요약**

| 분석 목적                     | 주요 컬럼                                             |
| :------------------------ | :------------------------------------------------ |
| **촬영 시간 순 타임라인 생성**       | `CapturedTime`, `Latitude`, `Longitude`           |
| **지도 시각화 / 이동 경로 추적**     | `Latitude`, `Longitude`, `CapturedTime`           |
| **카메라별 품질 비교**            | `Make`, `Model`, `ExposureTime`, `FNumber`, `ISO` |
| **위치 기반 라벨링 / 추천 시스템 입력** | `Latitude`, `Longitude`                           |
| **사진 관리 / 백업**            | `FileName`, `CapturedTime`, `Directory`           |

---

즉,

* `SourceFile`~`ISO` → **기기·촬영 세팅 기록**
* `Latitude`~`GPSAltitude` → **공간 정보**
* `CapturedTime` → **시간 축 기준**

이 세 영역이 조합되어, 이후 단계(주소 변환 → 지도 시각화 → AI 요약)에 쓰입니다.
